# ADNI Dataset Reshaping & Image Merge

In [1]:
import torch
import pandas as pd
import numpy as np
from glob import glob
import os, os.path
import matplotlib.pyplot as plt
from numpy import asarray

import datetime, os

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

import time
import datetime
import pickle

In [9]:
pd.options.display.max_columns = None

In [10]:
# Define a simple imputation function (can use mean, median, etc.)
def impute_patient_data(patient_data):
    numeric_cols = patient_data.select_dtypes(include=['float64', 'int64']).columns
    
    # Impute missing values by filling with the mean of the column
    patient_data[numeric_cols] = patient_data[numeric_cols].fillna(patient_data[numeric_cols].mean())
    return patient_data

# Load Dataset

In [11]:
original_adset = pd.read_csv('ADNIMERGE.csv')

# original_adset['IMAGEUID'] = original_adset['IMAGEUID'].astype(int)

C:\Users\myh04\AppData\Local\Temp\ipykernel_16656\2786337334.py:1: DtypeWarning: Columns (18,19,20,103,104) have mixed types. Specify dtype option on import or set low_memory=False.
  original_adset = pd.read_csv('ADNIMERGE.csv')


In [12]:
original_adset

,RID,PTID,VISCODE,SITE,COLPROT,ORIGPROT,EXAMDATE,DX_bl,AGE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,PTMARRY,APOE4,FDG,PIB,AV45,ABETA,TAU,PTAU,CDRSB,ADAS11,ADAS13,ADASQ4,MMSE,RAVLT_immediate,RAVLT_learning,RAVLT_forgetting,RAVLT_perc_forgetting,LDELTOTAL,DIGITSCOR,TRABSCOR,FAQ,MOCA,EcogPtMem,EcogPtLang,EcogPtVisspat,EcogPtPlan,EcogPtOrgan,EcogPtDivatt,EcogPtTotal,EcogSPMem,EcogSPLang,EcogSPVisspat,EcogSPPlan,EcogSPOrgan,EcogSPDivatt,EcogSPTotal,FLDSTRENG,FSVERSION,IMAGEUID,Ventricles,Hippocampus,WholeBrain,Entorhinal,Fusiform,MidTemp,ICV,DX,mPACCdigit,mPACCtrailsB,EXAMDATE_bl,CDRSB_bl,ADAS11_bl,ADAS13_bl,ADASQ4_bl,MMSE_bl,RAVLT_immediate_bl,RAVLT_learning_bl,RAVLT_forgetting_bl,RAVLT_perc_forgetting_bl,LDELTOTAL_BL,DIGITSCOR_bl,TRABSCOR_bl,FAQ_bl,mPACCdigit_bl,mPACCtrailsB_bl,FLDSTRENG_bl,FSVERSION_bl,Ventricles_bl,Hippocampus_bl,WholeBrain_bl,Entorhinal_bl,Fusiform_bl,MidTemp_bl,ICV_bl,MOCA_bl,EcogPtMem_bl,EcogPtLang_bl,EcogPtVisspat_bl,EcogPtPlan_bl,EcogPtOrgan_bl,EcogPtDivatt_bl,EcogPtTotal_bl,EcogSPMem_bl,EcogSPLang_bl,EcogSPVisspat_bl,EcogSPPlan_bl,EcogSPOrgan_bl,EcogSPDivatt_bl,EcogSPTotal_bl,ABETA_bl,TAU_bl,PTAU_bl,FDG_bl,PIB_bl,AV45_bl,Years_bl,Month_bl,Month,M,update_stamp
0,2,011_S_0002,bl,11,ADNI1,ADNI1,2005-09-08,CN,74.3,Male,16,Not Hisp/Latino,White,Married,0.0,1.36665,NaN,NaN,NaN,NaN,NaN,0.0,10.67,18.67,5.0,28.0,44.0,4.0,6.0,54.5455,10.0,34.0,112.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5 Tesla MRI,Cross-Sectional FreeSurfer (FreeSurfer Version...,35475.0,118233.0,8336.0,1229740.0,4177.0,16559.0,27936.0,1984660.0,CN,-4.37389,-4.179310,2005-09-08,0.0,10.67,18.67,5.0,28,44.0,4.0,6.0,54.5455,10.0,34.0,112.0,0.0,-4.37389,-4.179310,1.5 Tesla MRI,Cross-Sectional FreeSurfer (FreeSurfer Version...,118233.0,8336.0,1229740.0,4177.0,16559.0,27936.0,1984660.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.36665,NaN,NaN,0.000000,0.00000,0,0,2019-02-14 23:58:27.0
1,3,011_S_0003,bl,11,ADNI1,ADNI1,2005-09-12,AD,81.3,Male,18,Not Hisp/Latino,White,Married,1.0,1.08355,NaN,NaN,741.5,239.7,22.83,4.5,22.00,31.00,8.0,20.0,22.0,1.0,4.0,100.0000,2.0,25.0,148.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5 Tesla MRI,Cross-Sectional FreeSurfer (FreeSurfer Version...,32237.0,84599.0,5319.0,1129830.0,1791.0,15506.0,18422.0,1920690.0,Dementia,-16.61980,-16.206800,2005-09-12,4.5,22.00,31.00,8.0,20,22.0,1.0,4.0,100.0000,2.0,25.0,148.0,10.0,-16.61980,-16.206800,1.5 Tesla MRI,Cross-Sectional FreeSurfer (FreeSurfer Version...,84599.0,5319.0,1129830.0,1791.0,15506.0,18422.0,1920690.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,741.5,239.7,22.83,1.08355,NaN,NaN,0.000000,0.00000,0,0,2019-02-14 23:58:27.0
2,3,011_S_0003,m06,11,ADNI1,ADNI1,2006-03-13,AD,81.3,Male,18,Not Hisp/Latino,White,Married,1.0,1.05803,NaN,NaN,NaN,NaN,NaN,6.0,19.00,30.00,10.0,24.0,19.0,2.0,6.0,100.0000,NaN,19.0,135.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5 Tesla MRI,Cross-Sectional FreeSurfer (FreeSurfer Version...,31863.0,88580.0,5446.0,1100060.0,2427.0,14400.0,16972.0,1906430.0,Dementia,-15.04880,-13.421800,2005-09-12,4.5,22.00,31.00,8.0,20,22.0,1.0,4.0,100.0000,2.0,25.0,148.0,10.0,-16.61980,-16.206800,1.5 Tesla MRI,Cross-Sectional FreeSurfer (FreeSurfer Version...,84599.0,5319.0,1129830.0,1791.0,15506.0,18422.0,1920690.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,741.5,239.7,22.83,1.08355,NaN,NaN,0.498289,5.96721,6,6,2019-02-14 23:58:27.0
3,3,011_S_0003,m12,11,ADNI1,ADNI1,2006-09-12,AD,81.3,Male,18,Not Hisp/Latino,White,Married,1.0,1.09690,NaN,NaN,601.4,251.7,24.18,3.5,24.00,35.00,10.0,17.0,31.0,2.0,7.0,100.0000,0.0,21.0,126.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5 Tesla MRI,Cross-Sectional FreeSurfer (FreeSurfer Version...,35576.0,90099.0,5157.0,1095640.0,1596.0,14617.0,17330.0,1903820.0,Dementia,-21.46000,-20.271400,2005-09-12,4.5,22.00,31.00,8.0,20,22.0,1.0,4.0,100.0000,2.0,25.0,148.0,10.0,-16.61980,-16.206800,1.5 

# Check for Images

In [13]:
# Define the directory where the images are stored
image_dir = '/home/mason/ADNI_Dataset/ADNI_IMG_32.5%_x'

# Function to check if an image exists for a given patient_id
def get_image_path(image_id):
    try:
        image_id = int(image_id)
    except:
        image_id = image_id
        
    # Construct the image file path
    image_path = os.path.join(image_dir, f'I{image_id}.png')
    # Check if the file exists
    if os.path.exists(image_path):
        return image_path  # Return the path if it exists
    else:
        return np.nan  # Return None if the image does not exist

# Apply the function to each row of the dataset and create a new column 'image_path'
original_adset['image_path'] = original_adset['IMAGEUID'].apply(get_image_path)

In [14]:
# Now df has a new column 'image_path' with the image file path or None
original_adset

,RID,PTID,VISCODE,SITE,COLPROT,ORIGPROT,EXAMDATE,DX_bl,AGE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,PTMARRY,APOE4,FDG,PIB,AV45,ABETA,TAU,PTAU,CDRSB,ADAS11,ADAS13,ADASQ4,MMSE,RAVLT_immediate,RAVLT_learning,RAVLT_forgetting,RAVLT_perc_forgetting,LDELTOTAL,DIGITSCOR,TRABSCOR,FAQ,MOCA,EcogPtMem,EcogPtLang,EcogPtVisspat,EcogPtPlan,EcogPtOrgan,EcogPtDivatt,EcogPtTotal,EcogSPMem,EcogSPLang,EcogSPVisspat,EcogSPPlan,EcogSPOrgan,EcogSPDivatt,EcogSPTotal,FLDSTRENG,FSVERSION,IMAGEUID,Ventricles,Hippocampus,WholeBrain,Entorhinal,Fusiform,MidTemp,ICV,DX,mPACCdigit,mPACCtrailsB,EXAMDATE_bl,CDRSB_bl,ADAS11_bl,ADAS13_bl,ADASQ4_bl,MMSE_bl,RAVLT_immediate_bl,RAVLT_learning_bl,RAVLT_forgetting_bl,RAVLT_perc_forgetting_bl,LDELTOTAL_BL,DIGITSCOR_bl,TRABSCOR_bl,FAQ_bl,mPACCdigit_bl,mPACCtrailsB_bl,FLDSTRENG_bl,FSVERSION_bl,Ventricles_bl,Hippocampus_bl,WholeBrain_bl,Entorhinal_bl,Fusiform_bl,MidTemp_bl,ICV_bl,MOCA_bl,EcogPtMem_bl,EcogPtLang_bl,EcogPtVisspat_bl,EcogPtPlan_bl,EcogPtOrgan_bl,EcogPtDivatt_bl,EcogPtTotal_bl,EcogSPMem_bl,EcogSPLang_bl,EcogSPVisspat_bl,EcogSPPlan_bl,EcogSPOrgan_bl,EcogSPDivatt_bl,EcogSPTotal_bl,ABETA_bl,TAU_bl,PTAU_bl,FDG_bl,PIB_bl,AV45_bl,Years_bl,Month_bl,Month,M,update_stamp,image_path
0,2,011_S_0002,bl,11,ADNI1,ADNI1,2005-09-08,CN,74.3,Male,16,Not Hisp/Latino,White,Married,0.0,1.36665,NaN,NaN,NaN,NaN,NaN,0.0,10.67,18.67,5.0,28.0,44.0,4.0,6.0,54.5455,10.0,34.0,112.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5 Tesla MRI,Cross-Sectional FreeSurfer (FreeSurfer Version...,35475.0,118233.0,8336.0,1229740.0,4177.0,16559.0,27936.0,1984660.0,CN,-4.37389,-4.179310,2005-09-08,0.0,10.67,18.67,5.0,28,44.0,4.0,6.0,54.5455,10.0,34.0,112.0,0.0,-4.37389,-4.179310,1.5 Tesla MRI,Cross-Sectional FreeSurfer (FreeSurfer Version...,118233.0,8336.0,1229740.0,4177.0,16559.0,27936.0,1984660.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.36665,NaN,NaN,0.000000,0.00000,0,0,2019-02-14 23:58:27.0,NaN
1,3,011_S_0003,bl,11,ADNI1,ADNI1,2005-09-12,AD,81.3,Male,18,Not Hisp/Latino,White,Married,1.0,1.08355,NaN,NaN,741.5,239.7,22.83,4.5,22.00,31.00,8.0,20.0,22.0,1.0,4.0,100.0000,2.0,25.0,148.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5 Tesla MRI,Cross-Sectional FreeSurfer (FreeSurfer Version...,32237.0,84599.0,5319.0,1129830.0,1791.0,15506.0,18422.0,1920690.0,Dementia,-16.61980,-16.206800,2005-09-12,4.5,22.00,31.00,8.0,20,22.0,1.0,4.0,100.0000,2.0,25.0,148.0,10.0,-16.61980,-16.206800,1.5 Tesla MRI,Cross-Sectional FreeSurfer (FreeSurfer Version...,84599.0,5319.0,1129830.0,1791.0,15506.0,18422.0,1920690.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,741.5,239.7,22.83,1.08355,NaN,NaN,0.000000,0.00000,0,0,2019-02-14 23:58:27.0,NaN
2,3,011_S_0003,m06,11,ADNI1,ADNI1,2006-03-13,AD,81.3,Male,18,Not Hisp/Latino,White,Married,1.0,1.05803,NaN,NaN,NaN,NaN,NaN,6.0,19.00,30.00,10.0,24.0,19.0,2.0,6.0,100.0000,NaN,19.0,135.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5 Tesla MRI,Cross-Sectional FreeSurfer (FreeSurfer Version...,31863.0,88580.0,5446.0,1100060.0,2427.0,14400.0,16972.0,1906430.0,Dementia,-15.04880,-13.421800,2005-09-12,4.5,22.00,31.00,8.0,20,22.0,1.0,4.0,100.0000,2.0,25.0,148.0,10.0,-16.61980,-16.206800,1.5 Tesla MRI,Cross-Sectional FreeSurfer (FreeSurfer Version...,84599.0,5319.0,1129830.0,1791.0,15506.0,18422.0,1920690.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,741.5,239.7,22.83,1.08355,NaN,NaN,0.498289,5.96721,6,6,2019-02-14 23:58:27.0,NaN
3,3,011_S_0003,m12,11,ADNI1,ADNI1,2006-09-12,AD,81.3,Male,18,Not Hisp/Latino,White,Married,1.0,1.09690,NaN,NaN,601.4,251.7,24.18,3.5,24.00,35.00,10.0,17.0,31.0,2.0,7.0,100.0000,0.0,21.0,126.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5 Tesla MRI,Cross-Sectional FreeSurfer (FreeSurfer Version...,35576.0,90099.0,5157.0,1095640.0,1596.0,14617.0,17330.0,1903820.0,Dementia,-21.46000,-20.271400,2005-09-12,4.5,22.00,31.00,8.0,20,22.0,1.0,4.0,100.0000,2.0,25.0,148.0,10.0,-1

# Final Clean Up, Drop NaN Cohorts

In [15]:
original_adset.dropna(subset=["image_path"], inplace=True)
# original_adset.dropna(subset=["ADAS11", "ADAS13", "ADASQ4"], inplace=True)

original_adset

,RID,PTID,VISCODE,SITE,COLPROT,ORIGPROT,EXAMDATE,DX_bl,AGE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,PTMARRY,APOE4,FDG,PIB,AV45,ABETA,TAU,PTAU,CDRSB,ADAS11,ADAS13,ADASQ4,MMSE,RAVLT_immediate,RAVLT_learning,RAVLT_forgetting,RAVLT_perc_forgetting,LDELTOTAL,DIGITSCOR,TRABSCOR,FAQ,MOCA,EcogPtMem,EcogPtLang,EcogPtVisspat,EcogPtPlan,EcogPtOrgan,EcogPtDivatt,EcogPtTotal,EcogSPMem,EcogSPLang,EcogSPVisspat,EcogSPPlan,EcogSPOrgan,EcogSPDivatt,EcogSPTotal,FLDSTRENG,FSVERSION,IMAGEUID,Ventricles,Hippocampus,WholeBrain,Entorhinal,Fusiform,MidTemp,ICV,DX,mPACCdigit,mPACCtrailsB,EXAMDATE_bl,CDRSB_bl,ADAS11_bl,ADAS13_bl,ADASQ4_bl,MMSE_bl,RAVLT_immediate_bl,RAVLT_learning_bl,RAVLT_forgetting_bl,RAVLT_perc_forgetting_bl,LDELTOTAL_BL,DIGITSCOR_bl,TRABSCOR_bl,FAQ_bl,mPACCdigit_bl,mPACCtrailsB_bl,FLDSTRENG_bl,FSVERSION_bl,Ventricles_bl,Hippocampus_bl,WholeBrain_bl,Entorhinal_bl,Fusiform_bl,MidTemp_bl,ICV_bl,MOCA_bl,EcogPtMem_bl,EcogPtLang_bl,EcogPtVisspat_bl,EcogPtPlan_bl,EcogPtOrgan_bl,EcogPtDivatt_bl,EcogPtTotal_bl,EcogSPMem_bl,EcogSPLang_bl,EcogSPVisspat_bl,EcogSPPlan_bl,EcogSPOrgan_bl,EcogSPDivatt_bl,EcogSPTotal_bl,ABETA_bl,TAU_bl,PTAU_bl,FDG_bl,PIB_bl,AV45_bl,Years_bl,Month_bl,Month,M,update_stamp,image_path


In [16]:
cleaned_adset = original_adset[['RID', 'PTID', 'DX', 'MMSE', 'AGE', 'PTGENDER', 'PTEDUCAT', 'PTETHCAT', 
                                'PTRACCAT', 'PTMARRY', 'ADAS11', 'ADAS13', 'ADASQ4',
                                'Years_bl', 'EXAMDATE', 'image_path']]

cleaned_adset.dropna(subset=["DX", "MMSE"], inplace=True)

C:\Users\myh04\AppData\Local\Temp\ipykernel_16656\1030119681.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_adset.dropna(subset=["DX", "MMSE"], inplace=True)


In [17]:
cleaned_adset = cleaned_adset.sort_values('EXAMDATE')

In [18]:
cleaned_adset = cleaned_adset.groupby('PTID').apply(impute_patient_data).reset_index(drop=True)

C:\Users\myh04\AppData\Local\Temp\ipykernel_16656\2109356615.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  cleaned_adset = cleaned_adset.groupby('PTID').apply(impute_patient_data).reset_index(drop=True)


In [19]:
# Encode MMSE Classification Values

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Apply the LabelEncoder to the 'condition' column (for class labels)
cleaned_adset['DX_encoded'] = label_encoder.fit_transform(cleaned_adset['DX'])

In [20]:
# ohe.transform(cleaned_adset[['DX']])

In [21]:
# Turn classification labels into one hot encoding

# cleaned_adset['DX_encoded'] = ohe.transform(cleaned_adset[['DX']]).tolist()

In [22]:
# Check for NaNs in each patient after imputation
# Identify patients who still have NaNs
patients_with_nans = cleaned_adset[cleaned_adset.isna().any(axis=1)]['PTID'].unique()

# Remove patients with NaNs from the dataset
cleaned_adset = cleaned_adset[~cleaned_adset['PTID'].isin(patients_with_nans)]

In [23]:
# # Categorize age into bins
# age_bins = [0, 60, 65, 70, 75, 80, 85, 90, 120]
# age_labels = ['0-60', '61-65', '66-70', '71-75', '76-80', '81-85', '86-90', '90+']
# cleaned_adset['age_group'] = pd.cut(cleaned_adset['AGE'], bins=age_bins, labels=age_labels, right=False)

# # Categorize PTEDUCAT into bins
# educat_bins = [0, 10, 12, 14, 16, 18, 20, 90]
# educat_labels = ['0-10', '11-12', '13-14', '15-16', '17-18', '19-20', '21+']
# cleaned_adset['educat_group'] = pd.cut(cleaned_adset['PTEDUCAT'], bins=educat_bins, 
#                                        labels=educat_labels, right=False)

In [24]:
# # Make sure age_group is a string.
# cleaned_adset['age_group'] = cleaned_adset['age_group'].astype(str)
# # cleaned_adset['PTEDUCAT'] = cleaned_adset['PTEDUCAT'].astype(str)

# # Apply one-hot encoding on selected columns
# df_one_hot = pd.get_dummies(cleaned_adset[['age_group', 'PTGENDER', 'educat_group', 
#                                            'PTETHCAT', 'PTRACCAT', 'PTMARRY']], dtype=float)

# # Combine one-hot encoded columns into a vector for each row
# cleaned_adset['one_hot_vector'] = df_one_hot.apply(lambda row: row.values.tolist(), axis=1)

In [25]:
# Convert categorical entries to integer indicies.
# Create a dictionary to store the encoders for each column
encoders = {}

# List of categorical columns to encode
categorical_columns = ['PTGENDER', 'PTETHCAT', 'PTRACCAT', 'PTMARRY']

# Encode each column separately
for col in categorical_columns:
    encoder = LabelEncoder()
    cleaned_adset[f'{col}_encoded'] = encoder.fit_transform(cleaned_adset[col])
    encoders[col] = encoder  # Store the encoder for future use

# Age and PTEDUCAT are continuous, so they are fed directly to the model.

In [26]:
cleaned_adset

,RID,PTID,DX,MMSE,AGE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,PTMARRY,ADAS11,ADAS13,ADASQ4,Years_bl,EXAMDATE,image_path,DX_encoded,PTGENDER_encoded,PTETHCAT_encoded,PTRACCAT_encoded,PTMARRY_encoded


In [27]:
cleaned_adset['DX_encoded'].value_counts()

Series([], Name: count, dtype: int64)

In [28]:
cleaned_adset.dtypes

RID                   int64
PTID                 object
DX                   object
MMSE                float64
AGE                 float64
PTGENDER             object
PTEDUCAT              int64
PTETHCAT             object
PTRACCAT             object
PTMARRY              object
ADAS11              float64
ADAS13              float64
ADASQ4              float64
Years_bl            float64
EXAMDATE             object
image_path          float64
DX_encoded          float64
PTGENDER_encoded    float64
PTETHCAT_encoded    float64
PTRACCAT_encoded    float64
PTMARRY_encoded     float64
dtype: object

In [29]:
# Get number of cohorts left in the study after processing.
cleaned_adset['PTID'].nunique()

0

In [30]:
# Calculate category sizes
category_sizes = cleaned_adset.nunique().to_dict()

print("Category Sizes:", category_sizes)

Category Sizes: {'RID': 0, 'PTID': 0, 'DX': 0, 'MMSE': 0, 'AGE': 0, 'PTGENDER': 0, 'PTEDUCAT': 0, 'PTETHCAT': 0, 'PTRACCAT': 0, 'PTMARRY': 0, 'ADAS11': 0, 'ADAS13': 0, 'ADASQ4': 0, 'Years_bl': 0, 'EXAMDATE': 0, 'image_path': 0, 'DX_encoded': 0, 'PTGENDER_encoded': 0, 'PTETHCAT_encoded': 0, 'PTRACCAT_encoded': 0, 'PTMARRY_encoded': 0}


In [31]:
demographics_columns = ["PTGENDER_encoded", "PTETHCAT_encoded", "PTRACCAT_encoded", "PTMARRY_encoded"]

# find max in each column
for col in demographics_columns:
    max_value = cleaned_adset[col].max()
    print(f"Max value in {col}: {max_value}")

Max value in PTGENDER_encoded: nan
Max value in PTETHCAT_encoded: nan
Max value in PTRACCAT_encoded: nan
Max value in PTMARRY_encoded: nan


In [32]:
# print(type(cleaned_adset['one_hot_vector'][0]))

### Calculate Class Weights

In [33]:
class_counts = cleaned_adset['DX_encoded'].value_counts()

print(class_counts)

Series([], Name: count, dtype: int64)


In [34]:
print(type(class_counts))
print(sum(class_counts.values))

<class 'pandas.core.series.Series'>
0


In [35]:
total_samples = sum(class_counts.values)
class_weights = {cls: total_samples / count for cls, count in class_counts.items()}

print(class_weights)  # Output: A dictionary of weights for each class

{}


In [36]:
weights = torch.tensor([class_weights[i] for i in range(len(class_counts))], dtype=torch.float)

torch.save(weights, 'class_weights.pt')

print(weights)

tensor([])


# Reshape ADNI Dataset by Patients

In [37]:
# Ensure that the 'patient_id' column is in your dataset
if 'PTID' not in cleaned_adset.columns:
    raise ValueError("The dataset must contain a 'PTID' column.")

# Create a directory to save individual patient files (optional)
output_dir = 'patients_csv'
os.makedirs(output_dir, exist_ok=True)

# Group the dataframe by patient_id
grouped = cleaned_adset.groupby('PTID')

# Iterate over each group and save it to a separate CSV file
for patient_id, patient_data in grouped:
    filename = os.path.join(output_dir, f'{patient_id}.pkl')
    patient_data.to_pickle(filename)  
    print(f'Saved {filename}')

# Test Exported Patient

In [38]:
test_patient = pd.read_pickle('patients_csv/002_S_0295.pkl')

test_patient

,RID,PTID,DX,MMSE,AGE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,PTMARRY,ADAS11,ADAS13,ADASQ4,Years_bl,EXAMDATE,image_path,DX_encoded,PTGENDER_encoded,PTETHCAT_encoded,PTRACCAT_encoded,PTMARRY_encoded
0,295,002_S_0295,CN,28.0,84.8,Male,18,Not Hisp/Latino,White,Married,3.00,4.00,1.0,0.00000,2006-05-09,/home/mason/ADNI_Dataset/ADNI_IMG_32.5%_x/I118...,0,1,1,2,1
1,295,002_S_0295,CN,28.0,84.8,Male,18,Not Hisp/Latino,White,Married,5.33,6.33,1.0,0.48460,2006-11-02,/home/mason/ADNI_Dataset/ADNI_IMG_32.5%_x/I118...,0,1,1,2,1
2,295,002_S_0295,CN,30.0,84.8,Male,18,Not Hisp/Latino,White,Married,4.67,5.67,1.0,1.04312,2007-05-25,/home/mason/ADNI_Dataset/ADNI_IMG_32.5%_x/I640...,0,1,1,2,1
3,295,002_S_0295,CN,29.0,84.8,Male,18,Not Hisp/Latino,White,Married,3.67,5.67,2.0,2.23956,2008-08-04,/home/mason/ADNI_Dataset/ADNI_IMG_32.5%_x/I123...,0,1,1,2,1
4,295,002_S_0295,CN,28.0,84.8,Male,18,Not Hisp/Latino,White,Married,3.67,6.67,3.0,2.99521,2009-05-07,/home/mason/ADNI_Dataset/ADNI_IMG_32.5%_x/I150...,0,1,1,2,1


In [39]:
# print(type(test_patient.loc[0, 'one_hot_vector']))
# print(type(test_patient['one_hot_vector'][0]))

# Generate Manifest File

In [40]:
directory = 'patients_csv'
pid_t = []
path_t = []

for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
#     print(f)
    
    if os.path.isfile(f):
        try:
            unpickled_patient = pd.read_pickle(f)
            pid = filename.split('.')[0]
            pid_t.append(pid)
            path_t.append(f)
        except:
            print("Failed to read", f)
            
print(len(pid_t), len(path_t))

382 382


In [41]:
ad_patients_df = pd.DataFrame(
    {'PTID': pid_t,
     'path': path_t
    })

# ad_patients_df['PTID'] = pd.to_numeric(ad_patients_df['PTID'], downcast='integer')

In [42]:
ad_patients_df

,PTID,path
0,002_S_0295,patients_csv\002_S_0295.pkl
1,002_S_0413,patients_csv\002_S_0413.pkl
2,002_S_0619,patients_csv\002_S_0619.pkl
3,002_S_0685,patients_csv\002_S_0685.pkl
4,002_S_0729,patients_csv\002_S_0729.pkl
...,...,...
377,137_S_0994,patients_csv\137_S_0994.pkl
378,137_S_1041,patients_csv\137_S_1041.pkl
379,137_S_1414,patients_csv\137_S_1414.pkl
380,941_S_1194,patients_csv\941_S_1194.pkl


In [43]:
ad_patients_df.to_csv('AD_Patient_Manifest.csv', index=False)